<br><br><b><font size=10> CONTINUOUS CONTROL</font></b>
#### <i>...implementation for Udacity Deep Reinforcement Learning 

### Initialize Imports for the notebook
This Notebook uses code from separate python files where most of the implementation is handled

In [1]:
import env as E
from buffers import ReplayBuffer, nStepBuffer
from agent import D4PG_Agent
import models

# from agent import Agent
#from get_args import get_args

import os.path
import time
import re
import sys

import torch
import matplotlib.pyplot as plt
import numpy as np
from unityagents import UnityEnvironment
from collections import deque
import torchvision.transforms as T

In [2]:
import multiprocessing as multi
multi.cpu_count()

16

### Manually declare an ARGS class for the notebook, to take the place of argparser in the command line.

In [3]:
class Args:
    def __init__(self):
        self.train = True
        self.nographics = False
        self.num_eps = 10
        self.rollout = 5
        self.batchsize = 64
        self.pretrain = 1000
        
args = Args()

Confirm that the args are all set the way we want them.

In [4]:
for arg in vars(args):
    if arg == "sep": continue
    print("{}: {}".format(arg.upper(), getattr(args, arg)))

TRAIN: True
NOGRAPHICS: False
NUM_EPS: 10
ROLLOUT: 5
BATCHSIZE: 64
PRETRAIN: 1000


### Load the environment and print a bit of information contained in the wrapper class

In [5]:
env = E.Environment(args)
print("State size:", env.state_size)
print("Action size:", env.action_size)
print("Num Agents:", env.agent_count)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


State size: 33
Action size: 4
Num Agents: 20


In [6]:
env.train = False
env.reset()

In [ ]:
import importlib
import agent
import buffers
importlib.reload(agent)
importlib.reload(buffers)
importlib.reload(models)

In [ ]:
# agent = agent.D4PG_Agent(env.state_size, env.action_size, env.agent_count, args.rollout)
# print(agent.__class__.__name__)
# print(agent.memory)
# agent.initialize_memory(10, env)

In [9]:
states = env.states
states.shape

(20, 33)

### Test out Agent actions without training

In [7]:
m = models.ActorNet(env.state_size, env.action_size)

In [10]:
# Generate a test action
a= m(torch.from_numpy(states).float()).detach()
print(a.shape)
a = a.numpy()
print(a.shape)
print(type(a))

torch.Size([20, 4])
(20, 4)
<class 'numpy.ndarray'>


In [11]:
a = np.random.randn(env.agent_count, env.action_size)
print(a.shape)
print(type(a))


(20, 4)
<class 'numpy.ndarray'>


In [12]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(200):
    actions = m(torch.from_numpy(states).float()).detach().numpy()
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

Total score (averaged over agents) this episode: 0.11799999736249447


### Take random actions in the environment below 
<i>
-to check that code is working<br>
-to get familiar with the environment

In [ ]:
scores = np.zeros(env.agent_count)
states = env.states
for i in range(200):
    actions = np.random.randn(env.agent_count, env.action_size)
    actions = np.clip(actions, -1, 1)
    rewards, dones = env.step(actions)
    scores += rewards
    states = env.states
    if np.any(dones):
        break
    i += 1
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))    

In [ ]:
env.close()